# Chuẩn hóa cơ sở dữ liệu – Chuẩn thứ hai (2NF)


## Câu 1. Xác định khóa chính của bảng AgentTasks_Raw.
### Answer:
- Bảng `AgentTask_Raw` này có 2 khóa chính là `agent_id` và `task_id`

## Câu 2. Bảng này có vi phạm 2NF không? Nếu có, chỉ rõ thuộc tính nào phụ thuộc từng phần.
### Answer:
- Bảng này có vi phạm 2NF vì:
  - khóa chính là: (`agent_id` và `task_id`)
  - (`agent_name` và `llm_model`) chỉ phụ thuộc `agent_id`
  - (`task_name` và `task_type`) chỉ phụ thuộc `task_id`
 

## Câu 3. Hãy chuẩn hoá bảng trên về chuẩn 2NF.
### Answer:
- Bảng sau khi chuẩn hóa được chia làm 3 bảng:
  - Bảng `Agents`(`agent_id`, `agent_name`, `llm_model`)
  - Bảng `Tasks` (`task_id`, `task_name`, `task_type`)
  - Bảng `AgentTasks`(`agent_id`, `task_id`)


## Câu 4. Mô tả các bảng sau chuẩn hoá: tên bảng, các cột, khóa chính và khóa ngoại.
### Answer:
- Bảng `Agents`:
  - agent_id VARCHAR(10) PRIMARY KEY,
  - agent_name VARCHAR(100) NOT NULL,
  - llm_model VARCHAR(100) NOT NULL
- Bảng `Tasks`:
  - task_id VARCHAR(10) PRIMARY KEY,
  - task_name VARCHAR(100) NOT NULL,
  - task_type VARCHAR(20) NOT NULL
- Bảng `AgentTasks`:
  - agent_id INT,
  - task_id INT,
  - FOREIGN KEY (agent_id) REFERENCES Agents(agent_id) ON DELETE CASCADE,
  - FOREIGN KEY (task_id) REFERENCES Tasks(task_id) ON DELETE CASCADE

## Câu 5. Câu hỏi truy vấn:
- Lấy danh sách các tác vụ và tên Agent thực hiện.
- Lấy danh sách các mô hình LLM đang được sử dụng.
- Đếm số tác vụ thuộc mỗi loại (task_type).
- Tìm các Agent sử dụng LLM có tên bắt đầu bằng chữ G

In [23]:
import pymysql
import os
from dotenv import load_dotenv

# Load environment variables from .credential file
load_dotenv("../../w7/.credential")

# Get credentials from environment variables
conn = pymysql.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)
cursor = conn.cursor()


In [ ]:
# # Drop existing tables if they exist
# cursor.execute("DROP TABLE IF EXISTS AgentTasks;")
# cursor.execute("DROP TABLE IF EXISTS Agents;")
# cursor.execute("DROP TABLE IF EXISTS Tasks;")

0

In [27]:
cursor.execute("CREATE DATABASE IF NOT EXISTS w9db;")
cursor.execute("USE w9db;")

# 1. Tạo bảng Agents
cursor.execute("""
CREATE TABLE IF NOT EXISTS Agents (
    agent_id VARCHAR(10) PRIMARY KEY,
    agent_name VARCHAR(100) NOT NULL,
    llm_model VARCHAR(100) NOT NULL
);
""")

# 2. Tạo bảng Tasks
cursor.execute("""
CREATE TABLE IF NOT EXISTS Tasks (
    task_id VARCHAR(10) PRIMARY KEY,
    task_name VARCHAR(100) NOT NULL,
    task_type VARCHAR(20) NOT NULL
);
""")

# 3. Tạo bảng AgentTasks
cursor.execute("""
CREATE TABLE IF NOT EXISTS AgentTasks (
    agent_id VARCHAR(10),
    task_id VARCHAR(10),
    PRIMARY KEY (agent_id, task_id),
    FOREIGN KEY (agent_id) REFERENCES Agents(agent_id) ON DELETE CASCADE,
    FOREIGN KEY (task_id) REFERENCES Tasks(task_id) ON DELETE CASCADE
);
""")

# 4. Chèn dữ liệu vào bảng Agents
cursor.execute("""
INSERT INTO Agents (agent_id, agent_name, llm_model) VALUES
('A01', 'Claude Agent', 'Claude 3'),
('A02', 'GPT Assistant', 'GPT-4'),
('A01', 'Claude Agent', 'Claude 3'),
('A03', 'Gemini Helper', 'Gemini 1.5')
ON DUPLICATE KEY UPDATE agent_name=VALUES(agent_name), llm_model=VALUES(llm_model);
""")

# 5. Chèn dữ liệu vào bảng Tasks
cursor.execute("""
INSERT INTO Tasks (task_id, task_name, task_type) VALUES
('T01', 'Summarization', 'NLP'),
('T02', 'SQL Generation', 'Code'),
('T03', 'Sentiment Analysis', 'NLP'),
('T01', 'Summarization', 'NLP')
ON DUPLICATE KEY UPDATE task_name=VALUES(task_name), task_type=VALUES(task_type);
""")

# 6. Chèn dữ liệu vào bảng AgentTasks
cursor.execute("""
INSERT INTO AgentTasks (agent_id, task_id) VALUES
('A01', 'T01'),
('A02', 'T02'),
('A01', 'T03'),
('A03', 'T01')
""")

# Commit the changes
conn.commit()

In [18]:
# Lấy danh sách các tác vụ và tên Agent thực hiện
cursor.execute("""
SELECT a.agent_name, t.task_name 
FROM agents a
INNER JOIN agenttasks at
    ON a.agent_id = at.agent_id
INNER JOIN tasks t
    ON t.task_id = at.task_id;
""")

data = cursor.fetchall()
print(data)

(('Claude Agent', 'Summarization'), ('Claude Agent', 'Sentiment Analysis'), ('GPT Assistant', 'SQL Generation'), ('Gemini Helper', 'Summarization'))


In [19]:
# Lấy danh sách các mô hình LLM đang được sử dụng.
cursor.execute("""
SELECT llm_model
FROM agents;
""")

data = cursor.fetchall()
print(data)

(('Claude 3',), ('GPT-4',), ('Gemini 1.5',))


In [20]:
# Đếm số tác vụ thuộc mỗi loại (task_type)
cursor.execute("""
SELECT task_type, count(*) 
    AS task_count
FROM tasks
GROUP BY task_type;
""")

data = cursor.fetchall()
print(data)

(('NLP', 2), ('Code', 1))


In [21]:
# Tìm các Agent sử dụng LLM có tên bắt đầu bằng chữ G.

cursor.execute("""
SELECT agent_name 
FROM agents
WHERE agent_name like "G%";
""")

data = cursor.fetchall()
print(data)

(('GPT Assistant',), ('Gemini Helper',))


In [28]:
# Close Connection
cursor.close()
conn.close()